# 

****

 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```

1. Testing my solution by running the cells following your block (indicated by ##TEST##)

**Notes**

---

#Decision Trees

---
** Setup **

In [1]:
require 'test/unit/assertions'
require 'daru'
require 'distribution'
require 'json'

include Test::Unit::Assertions

## Loads data files
def read_sparse_data_from_csv prefix
  data = []
  classes = Hash.new {|h,k| h[k] = 0}
  header = File.read(prefix + ".header").chomp.split(",")  
  
  File.open(prefix + ".csv").each_line.with_index do |l, i|
    a = l.chomp.split ","
    next if a.empty?
    row = {"features" => Hash.new}
    
    header.each.with_index do |k, i|
      v = a[i].to_f
      if k == "label"
        row["label"] = v.to_i
      else
        next if v.zero?
        row["features"][k] = v
      end
    end
    classes[row["label"]] += 1
    
    data << row
  end
  return {"classes" => classes, "features" => header[0,header.size - 1], "data" => data}
end

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

:read_sparse_data_from_csv

In [2]:
#Preview 2 lines from the Iris dataset
iris = read_sparse_data_from_csv "iris"
iris["data"][0,2]

[{"features"=>{"sepal_length"=>7.0, "sepal_width"=>3.2, "petal_length"=>4.7, "petal_width"=>1.4}, "label"=>1}, {"features"=>{"sepal_length"=>5.7, "sepal_width"=>3.0, "petal_length"=>4.2, "petal_width"=>1.2}, "label"=>1}]

In [3]:
#Preview 2 lines from the Spambase dataset
spambase = read_sparse_data_from_csv "spambase"
spambase["data"][0,2]

[{"features"=>{"word_freq_our"=>0.27, "word_freq_mail"=>0.83, "word_freq_you"=>0.27, "word_freq_your"=>0.27, "word_freq_font"=>8.58, "char_freq_["=>0.092, "char_freq_$"=>0.185, "char_freq_#"=>0.232, "capital_run_length_average"=>7.313, "capital_run_length_longest"=>99.0, "capital_run_length_total"=>607.0}, "label"=>1}, {"features"=>{"word_freq_your"=>0.9, "word_freq_george"=>0.9, "word_freq_data"=>0.9, "char_freq_["=>0.14, "capital_run_length_average"=>3.472, "capital_run_length_longest"=>28.0, "capital_run_length_total"=>125.0}, "label"=>0}]

---
## Question 1.1 (10 Points)
Write a function that calculates the class distribution, $p(c)$, for all classes $c$ for a dataset. 

In [4]:
def class_distribution dataset
  classes = Hash.new {|h,k| h[k] = 0}
  dataset.each do |key,array|
    classes[key["label"]]=1+classes[key["label"]]
  end
  
  result={}
  classes.each do |key,array|
    result[key]=array.to_f/dataset.size.to_f
  end
  
  return result
end

:class_distribution

In [5]:
### TESTS ###
# Check that there are three classes
t1_iris_dist = class_distribution iris["data"]
t1_iris_num_classes = 3
assert_equal(t1_iris_num_classes, t1_iris_dist.size)

In [6]:
### TESTS ###
# Check that each class has a probability 1/3
t1_iris_dist = class_distribution iris["data"]
t1_iris_num_classes = 3
n = t1_iris_num_classes.times do |cls|
  assert_in_delta t1_iris_dist[cls], 0.33333, 1e-4
end
assert_equal t1_iris_num_classes, n

In [7]:
### TESTS ###
t1_spambase_dist = class_distribution spambase["data"]
t1_spambase_num_classes = 2
assert_equal(t1_spambase_num_classes, t1_spambase_dist.size)

In [8]:
### TESTS ###
t1_spambase_dist = class_distribution spambase["data"]
assert_in_delta t1_spambase_dist[0], 0.6, 1e-1
assert_in_delta t1_spambase_dist[1], 0.4, 1e-1

## Question 2 (20 Points)
using input from a hash table rather than an array. 

In [9]:
def entropy dist

  ent=0
  dist.each do |key,array|
    if array==0
      return 0.0
    end
    ent+=-array*Math.log(array)
  end
  
  ent
end

:entropy

In [10]:
### TESTS ###
#Checks the class entropy for Iris dataset
t2_iris_dist = class_distribution iris["data"]
t2_iris_entropy = entropy t2_iris_dist
assert_in_delta(1.0986122886681096, t2_iris_entropy, 1e-4)

In [11]:
### TESTS ###
#Checks the class entropy for Spambase dataset
t2_spambase_dist = class_distribution spambase["data"]
t2_spambase_entropy = entropy t2_spambase_dist
assert_in_delta(0.6705230209876485, t2_spambase_entropy, 1e-4)

In [12]:
### TESTS ###
# Checks that code handles a class with zero members of a class
t2_zero_dist = {0 => 0.0, 1 => 107.0}
t2_zero_entropy = entropy t2_zero_dist
assert_in_delta(0.0, t2_zero_entropy, 1e-4)

## Question 3.1 (5 points)

Implement information gain given an initial entropy $H_0$ and a hash of counts, where the key of each hash is a split criterion and the value is an array of all examples matching the split. Call the ```class_distribution``` and ```entropy``` functions you have defined above.

The formula for information gain is as follows:

## $IG(Q, V) = H_0 - \sum_{v\in V} \frac{|v|}{|V|} H(c\mid v) $ 

In [120]:
def information_gain h0, splits

  
  size = Hash.new {|h,k| h[k] = 0}
  sum = Hash.new {|h,k| h[k] = 0}
  total=0
  
  splits.each do |key, array|
    total+=array.size
  end
  
  splits.each do |key, array|
    sum[key]+=entropy(class_distribution(array))
    size[key]=array.size
  end
  
  result=0
  
  size.each do |key,array|
    size[key]=size[key].to_f/total
  end
  
  splits.each do |key, array|
    result+=sum[key]*size[key]
  end
  
  puts h0-result
  return h0-result
end

:information_gain

In [121]:
### TEST: Random Split ###
t3_random_split = iris["data"].group_by {|row| rand > 0.5 ? "l" : "r"}
t3_entropy = entropy(class_distribution(iris["data"]))


1.0986122886681096

In [122]:
def t3_random_information_gain_spambase
  iris = read_sparse_data_from_csv "iris"
  t3_random_split = iris["data"].group_by {|row| rand > 0.5 ? "l" : "r"}
  t3_entropy = entropy(class_distribution(iris["data"]))
  return information_gain t3_entropy, t3_random_split
end
t3_random_information_gain_spambase

0.0016151136677073907


0.0016151136677073907

## Question 3.2 (5 points)

The test above verifies the information gain when the iris data is split with uniform probability. What should the information gain be here if we were to do this for the **spambase** dataset?

In [123]:
## Make this function return a constant equal to your answer to the question above.
def t3_random_information_gain_spambase
  spambase = read_sparse_data_from_csv "spambase"
  t3_random_split = spambase["data"].group_by {|row| rand > 0.5 ? "l" : "r"}
  t3_entropy = entropy(class_distribution(spambase["data"]))
  return information_gain t3_entropy, t3_random_split
end

:t3_random_information_gain_spambase

In [124]:
### TEST (Hidden) ###
assert_true(t3_random_information_gain_spambase() >= 0)

0.00035707015949548726


## Question 3.3 (10 points)

The test below verifies the information gain of the iris data when split by class value. What should the information gain be here if we were to do this for the **spambase** dataset?

In [125]:
### TEST: Perfect split (by class)
t3_perfect_split = iris["data"].group_by {|row| row["label"]}
t3_perfect_information_gain = information_gain t3_entropy, t3_perfect_split

1.0986122886681096


1.0986122886681096

In [126]:
## Make this function return a constant equal to your answer to the question above.
def t3_perfect_information_gain_spambase
  spambase = read_sparse_data_from_csv "spambase"
  t3_perfect_split = spambase["data"].group_by {|row| row["label"]}
  t3_entropy = entropy(class_distribution(spambase["data"]))
  return information_gain t3_entropy, t3_perfect_split
end

:t3_perfect_information_gain_spambase

In [127]:
 t3_perfect_information_gain_spambase

0.6705230209876485


0.6705230209876485

In [128]:
### TEST (Hidden) ###

assert_true (t3_perfect_information_gain_spambase() > 0) 

0.6705230209876485


## Question 4 (20 points)
Considering the only the figure below, which shows the iris dataset colored by class, what value of petal_length would have the best information gain?

In [129]:
d = iris["data"]
df = Daru::DataFrame.new({
  x1: d.collect{|r| r["features"]["petal_length"]}, 
  x2: d.collect{|r| r["features"]["sepal_width"]}, 
  label: d.collect{|r| r["label"]}
}).to_category(:label).plot(type: :scatter, x: :x1, y: :x2, categorized: {by: :label, method: :color}) do |plot, diagram|
  plot.x_label iris["features"][2]
  plot.y_label iris["features"][1]
  plot.legend true
end

#<Nyaplot::Plot:0x0000000004564ba0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x0000000004514588 @properties={:type=>:scatter, :options=>{:x=>:x1, :y=>:x2, :title=>1, :color=>"rgb(251,180,174)", :tooltip_contents=>[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}, :data=>"4986666f-2b24-4812-b244-6c7f1be74c26"}, @xrange=[3.0, 5.1], @yrange=[2.0, 3.4]>, #<Nyaplot::Diagram:0x00000000044cf6e0 @properties={:type=>:scatter, :options=>{:x=>:x1, :y=>:x2, :title=>2, :color=>"rgb(179,205,227)", :tooltip_contents=>[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]}, :data=>"ed4b02d6-3cd0-4bcd-9651-6bd0010b0490"}, @xrange=[4.5, 6.9], @yrange=[2.2, 3.8]>, #<Nyaplot::Diagram:0x0000000004481eb8 @properties={:type=>:scatter, :options=>{:x=>:x1, :y=>:x2, :title=>0, :color=>"rgb(204,235,197)", :tooltip_contents=>[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, :data=>"a0a2b94b-7102-4032-b05c-a3b187522b77"}, @xrange=[1.0, 1.9], @yrange=[2.3, 4.4]>], :options=>{:legend=>true, :x_label=>"petal_length", :y_label=>"sepal_width", :zoom=>true, :width=>800, :xrange=>[1.0, 6.9], :yrange=>[2.0, 4.4]}}>

In [130]:
## Make this function return a constant equal to your answer to the question above.
def t4_best_split_for_petal_length_guess
    return 2
end

:t4_best_split_for_petal_length_guess

In [131]:
### TEST (Hidden) ###

## Question 5 (20 points)
Implement a function which takes a feature name and a real-valued threshold and splits data into two groups: 
* Strictly less than the threshold
* Greater than or equal to the threholds. 

This function returns a structure that can be used to calculate the information gain. An example is as follows:
```
{"petal_length < X1":[
        {"features":{"sepal_length":4.8,"sepal_width":3.1,"petal_length":1.6,"petal_width":0.2},"label":0},
],"petal_length >= X1":[
        {"features":{"sepal_length":4.8,"sepal_width":3.1,"petal_length":1.6,"petal_width":0.2},"label":0},
]}
```

In [132]:
def split_on_numeric_value x, k, v
  splits={"less than"=>[], "more than"=>[]}

  x.each do |item|
    if item["features"][k]
      y=item["features"][k]
    else
      y=0
    end
    if y<v
      splits["less than"]+=[item]
    end
    if y>=v
      splits["more than"]+=[item]
    end
  end
  
  splits
end

:split_on_numeric_value

In [133]:
### TEST ###

# Checks for the number of examples when split on petal_length = 1.7
t5_iris_splits = split_on_numeric_value iris["data"], "petal_length", 1.7
t5_split_sizes = t5_iris_splits.values.collect {|v| v.size}.sort
t5_num_splits = 2
assert_equal t5_num_splits, t5_split_sizes.size
assert_equal 44, t5_split_sizes[0]
assert_equal 106, t5_split_sizes[1]

In [134]:
### TEST ###

# Checks the information gain for this split
t5_iris_entropy = entropy(class_distribution(iris["data"]))
t5_iris_information_gain = information_gain t5_iris_entropy, t5_iris_splits
assert_in_delta 0.48280104455013506, t5_iris_information_gain, 5e-2

0.48280104455013506


In [135]:
### TEST ###

# Checks the information gain for a split on the spambase dataset
t5_spambase_splits = split_on_numeric_value spambase["data"], "char_freq_$", 0.056
t5_spambase_entropy = entropy(class_distribution(spambase["data"]))
t5_spambase_information_gain = information_gain t5_spambase_entropy, t5_spambase_splits
assert_in_delta 0.17012249631509135, t5_spambase_information_gain, 5e-2

0.17012249631509135


In [136]:
### TEST ###
sorted_values = iris["data"].collect {|r| r["features"]["petal_length"]}.uniq.sort
min_sl = sorted_values.first
max_sl = sorted_values.last

x = []
y = []
h0 = entropy(class_distribution(iris["data"]))
sorted_values.each do |t|
  x << t
  y << information_gain(h0, split_on_numeric_value(iris["data"], "petal_length", t))
end

assert_equal 43, sorted_values.size
Daru::DataFrame.new({x: x, y: y}).plot(type: :line, x: :x, y: :y) do |plot, diagram|
  plot.x_label "Petal Length Threshold"
  plot.y_label "Information Gain"
  plot.xrange [min_sl,max_sl]
end

0.0
0.007368925763855394
0.014829172330138585
0.030033875414081868
0.08654114224614062
0.19846652829937905
0.36762441317024375
0.48280104455013506
0.5708880749943686
0.6365141682948128
0.6083557734545781
0.5738298777087836
0.5487967703668577
0.5383976022416942
0.5290762982094618
0.5206770429611144
0.49998635614757925
0.4769733143948972
0.46858904960369663
0.4628103300330144
0.4620981203732968
0.4649602832666839
0.4499611899497512
0.46469597543888563
0.5021697266465265
0.4724565131247582
0.438117862765954
0.40314245075086563
0.32626067487019494
0.3005336653214238
0.2760652012017689
0.2527100690630446
0.21951214867965607
0.15864598561813148
0.13053245657705614
0.10373813181609481
0.08654114224614062
0.06983636023057316
0.045635817309936266
0.037783754878081055
0.030033875414081868
0.022383272446196978
0.007368925763855394


#<Nyaplot::Plot:0x00000000044bc360 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000000044f6c40 @properties={:type=>:line, :options=>{:x=>:x, :y=>:y}, :data=>"ab9749d9-9a35-4db7-b5f1-1a62ea4c187b"}, @xrange=[1.0, 6.9], @yrange=[0.0, 0.6365141682948128]>], :options=>{:x_label=>"Petal Length Threshold", :y_label=>"Information Gain", :xrange=>[1.0, 6.9], :zoom=>true, :width=>700, :yrange=>[0.0, 0.6365141682948128]}}>

## Question 6 (10 points)
Finding the best split value by calling ```split_on_numeric_value``` on each threshold is _very_ expensive. There is a better solution. The function should return an array consisting of the best threshold and the information gain of that threshold.


In order to find the best split, we need to find $t^\star$ defined as follows:

$ t^\star = \arg \max_{t} IG(x,V(t)) $

Given that a threshold is a single real value, we can simply calculate the values $IG(x,V)$ for every value of $t$ _in order_, which leads allows us to save some work by calculating information incrementally. Notice that the split sets have a specialized form, which we can write as $V(t) = \left\{v_L(t), v_R(t)\right\}$ with $v_L(t) = \left\{x \mid x < t \right\}$ and $v_R(t) = \left\{x \mid x \ge t \right\}$. 

As we sweep $t$ from minimum to maximum values, we note the following relations:

$v_R(t + \Delta_t) = v_R(t) - v_\Delta$

$v_L(t + \Delta_t) = v_L(t) \cup v_\Delta$

where $v_\Delta = \left\{x \mid t < x \le t + \Delta_t \right\}$

What we add to $v_L$ is exactly what was removed from $v_R$, which means that we can easily recalculate the information gain by incrementing a class-specific counter for $v_L$ and decrementing a class-specific counter for $v_R$ as we sweep through the range of the threshold.

The function below returns an array with the split point and the information gain:
```ruby
return [t_max, ig_max]
```


In [139]:
  def find_split_point_numeric examples, parent_entropy, feature_name  
    
    if(matches?(examples, feature_name))

      sorted_values = examples.collect {|r| r["features"][feature_name]}.uniq.sort
      ig_max=-1
      t_max=-1

      threshold = []
      iG = []
      sorted_values.each do |item|
        threshold << item
        split1 = NumericSplit.new(feature_name, item)
        iG << information_gain(parent_entropy, split1.split_on_feature(examples))
        if iG.last>ig_max
          ig_max=iG.last
          t_max=threshold.last
        end
      end

      split = NumericSplit.new feature_name, t_max
      ig=information_gain(parent_entropy, split.split_on_feature(examples))

      return {"split" => split, "information_gain" => ig}
    end   
  end

:find_split_point_numeric

In [140]:
### TEST ###
t6_iris_entropy = entropy(class_distribution(iris["data"]))
t6_split_sepal_width = find_split_point_numeric iris["data"], t6_iris_entropy, "sepal_width"
assert_in_delta 3.4, t6_split_sepal_width[0], 1e-2

149.0


In [141]:
### TEST ###
assert_in_delta 0.18570201019349364, t6_split_sepal_width[1], 1e-2

In [142]:
### TEST ###
t6_spambase_entropy = entropy(class_distribution(spambase["data"]))
t6_split_dollar = find_split_point_numeric spambase["data"], t6_spambase_entropy, "char_freq_$"
assert_in_delta 0.056, t6_split_dollar[0], 1e-2

2104.0
1400.0


In [143]:
### TEST ###
assert_in_delta 0.17012249631509135, t6_split_dollar[1], 1e-2

In [144]:
### TEST: Produces same result as split_on_numeric_value ###
t6_iris_expected_ig = information_gain(t6_iris_entropy, split_on_numeric_value(iris["data"], "sepal_width", 3.4))
assert_in_delta t6_iris_expected_ig, t6_split_sepal_width[1], 1e-2
t6_spambase_expected_ig = information_gain(t6_spambase_entropy, split_on_numeric_value(spambase["data"], "char_freq_$", 0.056))
assert_in_delta t6_spambase_expected_ig, t6_split_dollar[1], 1e-2

0.18570201019349364
0.17012249631509135
